# Домашнее задание № 10. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете. 
В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.

In [1]:
import torch
import torch.nn as nn
from datasets import load_dataset
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer
import logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel("INFO")

In [3]:
# just default index for ignore in CrossEntropy
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
# prompt templates
PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведены инструкции, объясняющие задание вместе с необходимым контекстом. \n"
        "Ответь в соответствии с заданием. Постарайся сделать это последовательно и полно. \n\n"
        "### Инструкция:\n{instruction}\n\n### Контекст:\n{input}\n\n### Ответ: "
    ),
    "prompt_no_input": (
        "Ниже приведены инструкции, объясняющие задание."
        "Ответь в соответствии с заданием. Постарайся сделать это последовательно и полно. \n\n"
        "### Инструкция:\n{instruction}\n\n### Ответ: "
    ),
}

In [4]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [5]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    # cat targets with outputs
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    # set up labels for text2text
    labels = copy.deepcopy(input_ids)
    # change label ids whithin source length to ignore during loss computation
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [6]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logger.warning("Loading data...")
        list_data_dict = load_dataset("IlyaGusev/ru_turbo_alpaca")["train"]

        logger.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logger.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        # pad with IGNORE INDEX as well till the end
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        # return dict with attension mask for padded input values
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [ ]:
model_name = "facebook/opt-125m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [8]:
train_dataset = SupervisedDataset(tokenizer=tokenizer,)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

Loading data...
/home/local/.local/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Formatting inputs...
Tokenizing inputs... This may take some time...

KeyboardInterrupt



In [ ]:
train_args = transformers.TrainingArguments(
    learning_rate=3e-5, 
    num_train_epochs=3,
    per_device_train_batch_size=4,
    # gradient_checkpointing=True,
    gradient_accumulation_steps=1,
    # fp16=True,
    evaluation_strategy='no',
    weight_decay=0.,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    save_strategy='no',
    logging_steps=1000,
    output_dir="opt125_instruct_ft"
)



In [ ]:
trainer = Trainer(model=model, 
                 tokenizer=tokenizer, 
                 args=train_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None, 
                 data_collator=data_collator)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('opt125_instruct_ft/model.pt')

In [7]:
def predict_for_instruction(instruct, text, model):  

    inputs = tokenizer([instruct.format(text)], 
                        return_tensors="pt", padding=True).to("cuda")

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=7,
        max_length=1024,
    # no_repeat_ngram_size=3, 
    repetition_penalty= 8.0,
    # length_penalty=0.01,
    #  early_stopping=True,
    do_sample=False, 
    # top_k=15, 
    # top_p=0.8, 
    # early_stopping=False,
    #     num_return_sequences=3,
    num_return_sequences=1,
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    )
    summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    return summaries[0]

С этой моделью скорее всего выйдет плохо, поскольку она плохо заточена под русский, а за обучение на инструкциях выучить достаточно она вряд ли могла бы. Зато можно попробовать погенерировать разный бред

In [11]:
# забыл сохранить модель, но оно все равно работало плохо, так что пусть разве что на память остается

In [ ]:
instruct = "Напиши, как сделать омлет."
text = ""


In [ ]:
predict_for_instruction(instruct, text, model)

In [ ]:
predict_for_instruction(instruct, text, model)

In [ ]:
instruct = "Предложи заголовок для текста ниже. Текст: {}"
text = """Исходная точка, для которой требуется интерпретация, выбирается из набора данных. 
Это может быть любой пример (например, строка данных для табличных данных, изображение, текст и т.д.), 
для которого было сделано предсказание сложной модели. Эта точка служит центром локального анализа и основой для генерации новых, 
возмущенных примеров данных. Возмущения создаются путем внесения небольших изменений в исходные данные. 
Эти изменения могут быть реализованы различными способами, в зависимости от типа данных
"""

In [ ]:
predict_for_instruction(instruct, text, model)

### try other model

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2",
                                                      max_len=512,
                                                      padding_side="right",
                                                      cache_dir="huggingface_cache",)

In [32]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
train_dataset = SupervisedDataset(tokenizer=tokenizer,)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

Loading data...
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Formatting inputs...
Tokenizing inputs... This may take some time...


In [10]:
model = transformers.AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2",
                                         cache_dir="huggingface_cache",)

In [11]:
train_args = transformers.TrainingArguments(
    learning_rate=5e-5,
    do_eval=False,
    do_predict=False,
    num_train_epochs=3,
    dataloader_num_workers=4,
    per_device_train_batch_size=4,
    gradient_checkpointing=True,
    gradient_accumulation_steps=8,
    evaluation_strategy='no',
    weight_decay=0.,
    warmup_ratio=0.1,
    optim="adafactor",
    lr_scheduler_type="cosine",
    save_strategy='no',
    logging_steps=200,
    output_dir="rugpt_instruct_ft"
)



In [13]:
trainer = Trainer(model=model, 
                 tokenizer=tokenizer, 
                 args=train_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None, 
                 data_collator=data_collator)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train()

wandb: Currently logged in as: xenomirant. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
200,2.430200
400,2.160500
600,2.098900
800,2.065700
1000,1.972500
1200,1.851500
1400,1.832900
1600,1.830900
1800,1.822700
2000,1.714600


TrainOutput(global_step=2796, training_loss=1.8876232643836899, metrics={'train_runtime': 3440.327, 'train_samples_per_second': 26.005, 'train_steps_per_second': 0.813, 'total_flos': 7684375133952000.0, 'train_loss': 1.8876232643836899, 'epoch': 3.0})

In [15]:
trainer.save_model('rugpt_instruct_ft/model.pt')

In [29]:
def predict_for_instruction(instruct, text, model):  

    inputs = tokenizer([instruct.format(text)], 
                        return_tensors="pt", padding=True).to("cuda")
    with torch.no_grad():
        
        output_sequences = model.generate(
            # this parameters are also important but you can read about them in the docs and just try changing them
            num_beams=7,
            max_length=1024,
        # no_repeat_ngram_size=3, 
        repetition_penalty= 8.0,
        length_penalty=0.01,
        #  early_stopping=True,
        do_sample=True, 
        # top_k=15, 
        # top_p=0.8, 
        early_stopping=True,
        #     num_return_sequences=3,
        num_return_sequences=1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        )
    summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    return summaries[0]

In [30]:
instruct = "Напиши, как сделать омлет."
text = ""


In [31]:
predict_for_instruction(instruct, text, model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'Напиши, как сделать омлет.  Для приготовления омлета нужно разогреть сковороду на среднем огне и добавить немного сливочного масла.  После того, как омлет готов, его можно подавать горячими или холодными. \n Омлет - это вкусное блюдо, которое всегда получается сытным и ароматным.  Он может быть приготовлен из различных ингредиентов, таких как яйца, молоко, сахар, мука, соль и т. д. Чтобы приготовить омлет, необходимо сначала нагреть сковороду до тех пор, пока она не станет золотисто-коричневого цвета.  Затем добавьте в сковороду мелко нарезанные овощи (например, помидоры, огурцы) и жарьте их с обеих сторон до золотистого цвета.  Когда омлет будет готов, посыпьте его сверху тертым пармезаном и украсьте свежими травами по вкусу. Приятного аппетита! ]]><img src="https://i.stack.imgur.com/t6R7k.png" border="0" width="600" height="450" /></a>'

In [33]:
predict_for_instruction(instruct, text, model)

'Напиши, как сделать омлет.  Чтобы приготовить омлет, нужно сначала разогреть сковороду на среднем огне и добавить немного оливкового масла.  Когда омлет будет готов, обжарьте его на сковороде до золотистой корочки.  Добавьте в сковороду мелко нарезанный лук и жарьте еще несколько минут.  Подавайте с рисом или сметаной.  Омлет получается очень вкусным! \n Оладьи из куриного мяса - это отличное блюдо для тех, кто хочет разнообразить свой рацион.  Для этого вам понадобятся: куринное филе, яйца, сыр пармезан, чеснок, соль и перец по вкусу.  Сначала сварите куриное филе в подсоленной воде до мягкости.  Разогрейте сковороду на среднем огне и добавьте небольшое количество сливочного масла.  Посыпьте яйцо солью и перцем по вкусу.  Нарежьте куриное филе на порции и выложите их на противень.  Поставьте запекаться в духовке при температуре 180 градусов Цельсия около 20-25 минут.  Готовые оладьи можно подавать со свежими фруктами или йогуртом.  Приятного аппетита! \n Яичница — это одно из самых л

In [36]:
instruct = "Предложи заголовок для текста ниже. Текст: {}"
text = """Графы и комплексы можно описывать признаковыми описаниями для вершин либо симплексов их составляющих – степенями, центральностями, коэффициентами кластеризации и т.д. Для графов существует признаковые описания основанные поиске графлетов – элементарных подграфов на небольшом (3-5) количестве вершин [Ribeiro2019]. Тогда каждая вершина графа описывается количеством ее вхождений в во все элементарные подграфы, а также количеством ее вхождения в эквивалентные до перестановки вершины этих подграфов, так называемые орбиты."""

In [37]:
predict_for_instruction(instruct, text, model)

'Предложи заголовок для текста ниже. Текст: Графы и комплексы можно описывать признаковыми описаниями для вершин либо симплексов их составляющих – степенями, центральностями, коэффициентами кластеризации и т.д. Для графов существует признаковые описания основанные поиске графлетов – элементарных подграфов на небольшом (3-5) количестве вершин [Ribeiro2019]. Тогда каждая вершина графа описывается количеством ее вхождений в во все элементарные подграфы, а также количеством ее вхождения в эквивалентные до перестановки вершины этих подграфов, так называемые орбиты.\nГрафы могут быть классифицированы по различным признакам, таким как цветность, размер шрифта, форма именительного паскаля и т.д. В зависимости от того, какие признаки обозначают вершину или комплекс, каждый признак может указывать на его принадлежность к какой-либо из категорий. Например, если у нас есть <<a href="#"><a href=""></a>, то мы можем использовать <<a href=""><a href="", чтобы найти <<a href=""><a href=""><a href=""><

In [38]:
instruct = "Посчитай, сколько ног у двух лошадей."
text = ""

In [39]:
predict_for_instruction(instruct, text, model)

'Посчитай, сколько ног у двух лошадей.\n\nВ общем, я не знаю, что делать с этими двумя лошадьми и почему они так сильно отличаются друг от друга? Может быть, это связано с тем, что две лошади имеют разный темперамент или как-то по-разному реагируют на разные условия окружающей среды? Я думаю, что лучше всего обратиться к ветеринару за помощью в поиске причины их различий.\n\nС уважением, [ваше мнение].\n[Ваше мнение о том, какие факторы влияют на различия в поведении этих двух лошадей.]\n[Твое мнение о том, какой фактор может повлиять на различие в поведении этой двух лошадей]\n[Твое мнение о том, каким образом эти две лошади могут влиять на поведение других лошадей]\n[Твое мнение о том, какое влияние оказывает разница в поведении обеих лошадей]\n[Твое мнение о том, какую роль играют изменения в поведении этих двух лошадей]\n[Твое мнение о том, какая роль играет изменение в поведении этих двух лошадей]'

Генерация сразу с рекламной интеграцией -- можно считать, готовая коммерческая модель.

In [40]:
instruct = "Расскажи, что стоит делать, чтобы лучше засыпать."
text = ""

In [41]:
predict_for_instruction(instruct, text, model)

'Расскажи, что стоит делать, чтобы лучше засыпать.  Что нужно сделать, чтобы улучшить качество сна? \n Для улучшения качества сна можно использовать различные методы: медитация, глубокое дыхание, йога, ароматерапия и т.д. Важно помнить, что сон - это не только физическое состояние, но и психологическое состояние, поэтому важно уделить внимание своему сну как можно больше времени для отдыха и релаксации.  Если вы хотите улучшить качество сна, обратитесь к специалистам за помощью или проконсультируйтесь у них по этому вопросу. \n Добавить комментарий Отменить ответ \n\n Вы можете войти на сайт, если вы зарегистрированы в одном из этих сервисов: \n Используйте вашу учетную запись VKontakte для входа на сайт. \n Используйте вашу учетную запись на Twitter.com для входа на сайт. \n Используйте вашу учетную запись на Facebook.com для входа на сайт. \n Используйте вашу учетную запись Google для входа на сайт. \n Используйте вашу учетную запись Мой Мир@Mail.ru для входа на сайт. \n Используйте 